In [1]:
from osgeo import gdal
from math import floor
import pandas as pd
import os

In [2]:
root_dir = 'D:\\OneDrive - vnu.edu.vn\\HK2 2021\\Các vấn đề\\map\\'
station_path = root_dir + 'station.csv'
station = pd.read_csv(station_path)
station.head()

,time,lat,lon,NO2,name,hpbl,press,rh,tmp,wspd,dpt,ndvi,omi_no2,pd,rd
0,2019-01-01,21.0491,105.8831,NaN,NVC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2019-01-02,21.0491,105.8831,NaN,NVC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2019-01-03,21.0491,105.8831,41.310400,NVC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2019-01-04,21.0491,105.8831,39.379204,NVC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2019-01-05,21.0491,105.8831,39.901879,NVC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**Format time**

In [3]:
def format_time(row):
    row.time = ''.join(row.time.split('-'))
    return row

In [4]:
station_formatted = station.apply(format_time, axis = 'columns')
station_formatted.head()

,time,lat,lon,NO2,name,hpbl,press,rh,tmp,wspd,dpt,ndvi,omi_no2,pd,rd
0,20190101,21.0491,105.8831,NaN,NVC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20190102,21.0491,105.8831,NaN,NVC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20190103,21.0491,105.8831,41.310400,NVC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20190104,21.0491,105.8831,39.379204,NVC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20190105,21.0491,105.8831,39.901879,NVC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**Extract feature**

In [5]:
def get_dpt_pixel_value(time, lat, lon):
    data_paths = []
    sum_pixel_value = 0
    for index in ['_00', '_06', '_12', '_18']:
        temp = root_dir + 'dpt\\DPT_' + str(time) + index +  '.tif'
        if os.path.exists(temp):
            data_paths.append(temp)
    for data_path in data_paths:
        ds = gdal.Open(data_path)
        gt = ds.GetGeoTransform()
        band = ds.GetRasterBand(1)
        mx = lon
        my = lat
        px = floor((mx - gt[0]) / gt[1]) #x pixel
        py = floor((my - gt[3]) / gt[5]) #y pixel
        pixel_value = band.ReadAsArray(px,py,1,1)[0][0]
        sum_pixel_value += pixel_value
    if len(data_paths) == 0:
        return
    return sum_pixel_value/len(data_paths)

In [6]:
def get_ndvi_pixel_value(time, lat, lon):
    """
    16 ngày đo 1 lần
    """
    date = str(time)
    month = date[4:6]
    data_paths = []
    sum_pixel_value = 0
    if month == '01':
        data_paths = [root_dir + 'ndvi\\MOD13Q1_20190101_Ndvi.tif', root_dir + 'ndvi\\MOD13Q1_20190117_Ndvi.tif']
    elif month == '02':
        data_paths = [root_dir + 'ndvi\\MOD13Q1_20190202_Ndvi.tif', root_dir + 'ndvi\\MOD13Q1_20190218_Ndvi.tif']
    elif month == '03':
        data_paths = [root_dir + 'ndvi\\MOD13Q1_20190306_Ndvi.tif', root_dir + 'ndvi\\MOD13Q1_20190322_Ndvi.tif']
    elif month == '04':
        data_paths = [root_dir + 'ndvi\\MOD13Q1_20190407_Ndvi.tif', root_dir + 'ndvi\\MOD13Q1_20190423_Ndvi.tif']
    elif month == '05':
        data_paths = [root_dir + 'ndvi\\MOD13Q1_20190509_Ndvi.tif', root_dir + 'ndvi\\MOD13Q1_20190525_Ndvi.tif']
    elif month == '06':
        data_paths = [root_dir + 'ndvi\\MOD13Q1_20190610_Ndvi.tif', root_dir + 'ndvi\\MOD13Q1_20190626_Ndvi.tif']
    elif month == '07':
        data_paths = [root_dir + 'ndvi\\MOD13Q1_20190712_Ndvi.tif', root_dir + 'ndvi\\MOD13Q1_20190728_Ndvi.tif']
    elif month == '08':
        data_paths = [root_dir + 'ndvi\\MOD13Q1_20190813_Ndvi.tif', root_dir + 'ndvi\\MOD13Q1_20190829_Ndvi.tif']
    elif month == '09':
        data_paths = [root_dir + 'ndvi\\MOD13Q1_20190914_Ndvi.tif', root_dir + 'ndvi\\MOD13Q1_20190930_Ndvi.tif']
    elif month == '10':
        data_paths = [root_dir + 'ndvi\\MOD13Q1_20191016_Ndvi.tif']
    elif month == '11':
        data_paths = [root_dir + 'ndvi\\MOD13Q1_20191101_Ndvi.tif', root_dir + 'ndvi\\MOD13Q1_20191117_Ndvi.tif']
    elif month == '12':
        data_paths = [root_dir + 'ndvi\\MOD13Q1_20191203_Ndvi.tif', root_dir + 'ndvi\\MOD13Q1_20191219_Ndvi.tif']
    for data_path in data_paths:
        ds = gdal.Open(data_path)
        gt = ds.GetGeoTransform()
        band = ds.GetRasterBand(1)
        mx = lon
        my = lat
        px = floor((mx - gt[0]) / gt[1]) #x pixel
        py = floor((my - gt[3]) / gt[5]) #y pixel
        pixel_value = band.ReadAsArray(px,py,1,1)[0][0]
        sum_pixel_value += pixel_value
    if len(data_paths) == 0:
        return
    return sum_pixel_value/len(data_paths)

In [7]:
def get_remaining_feature(feature, time, lat, lon):
    if feature == 'hpbl':
        data_path = root_dir + 'hpbl\\HPBLCombine_' + str(time) + '.tif'
    elif feature == 'press':
        data_path = root_dir + 'press\\PRESSCombine_' + str(time) + '.tif'
    elif feature == 'rh':
        data_path = root_dir + 'rh\\RHCombine_' + str(time) + '.tif'
    elif feature == 'tmp':
        data_path = root_dir + 'tmp\\TMPCombine_' + str(time) + '.tif'
    elif feature == 'wspd':
        data_path = root_dir + 'wspd\\WSPDCombine_' + str(time) + '.tif'
    elif feature == 'omi_no2':
        data_path = root_dir + 'omi_no2\\sen5p_' + str(time) + '.tif'
    elif feature == 'pd':
        data_path = root_dir + 'pd\\vnm_ppp_2019_resampled3km.tif'
    elif feature == 'rd':
        data_path = root_dir + 'rd\\road_dens.tif'
    if not os.path.exists(data_path):
        return
    ds = gdal.Open(data_path)
    gt = ds.GetGeoTransform()
    band = ds.GetRasterBand(1)
    mx = lon
    my = lat
    px = floor((mx - gt[0]) / gt[1]) #x pixel
    py = floor((my - gt[3]) / gt[5]) #y pixel
    pixel_value = band.ReadAsArray(px,py,1,1)[0][0]
    return pixel_value

In [8]:
def fill_feature(row):
    row.hpbl = get_remaining_feature('hpbl', row.time, row.lat, row.lon)
    row.press = get_remaining_feature('press', row.time, row.lat, row.lon)
    row.rh = get_remaining_feature('rh', row.time, row.lat, row.lon)
    row.tmp = get_remaining_feature('tmp', row.time, row.lat, row.lon)
    row.wspd = get_remaining_feature('wspd', row.time, row.lat, row.lon)
    row.dpt = get_dpt_pixel_value(row.time, row.lat, row.lon)
    row.ndvi = get_ndvi_pixel_value(row.time, row.lat, row.lon)
    row.omi_no2 = get_remaining_feature('omi_no2', row.time, row.lat, row.lon)
    row.pd = get_remaining_feature('pd', row.time, row.lat, row.lon)
    row.rd = get_remaining_feature('rd', row.time, row.lat, row.lon)
    return row

In [9]:
output = station_formatted.apply(fill_feature, axis = 'columns')

In [10]:
output.head()

,time,lat,lon,NO2,name,hpbl,press,rh,tmp,wspd,dpt,ndvi,omi_no2,pd,rd
0,20190101,21.0491,105.8831,NaN,NVC,391.863770,102794.367188,66.589279,11.087078,4.690219,5.061234,0.2078,NaN,27.722588,1828.877441
1,20190102,21.0491,105.8831,NaN,NVC,558.943237,102699.265625,65.773933,12.090031,3.590403,5.806078,0.2078,NaN,27.722588,1828.877441
2,20190103,21.0491,105.8831,41.310400,NVC,326.198273,102452.765625,69.259666,13.018890,2.969780,7.456484,0.2078,NaN,27.722588,1828.877441
3,20190104,21.0491,105.8831,39.379204,NVC,197.524628,102180.750000,78.663704,15.516078,2.775731,11.713000,0.2078,NaN,27.722588,1828.877441
4,20190105,21.0491,105.8831,39.901879,NVC,287.824951,102071.953125,75.866745,17.658155,3.128693,13.240313,0.2078,NaN,27.722588,1828.877441


In [11]:
#output.to_csv(root_dir + 'dataset.csv')